## Analysing H1B Visa data Trends

H1B visa is a nonimmigrant visa issued to gradaute level applicants allowing them to work in the United States. The employer sponsors the H1B visa for workers with theoretical or technical expertise in specialized fields such as in IT, finance, accounting etc. An interesting fact about immigrant workers is that about 52 percent of new Silicon valley companies were founded by such workers during 1995 and 2005. Some famous CEOs like Indira Nooyi (Pepsico), Elon Musk (Tesla), Sundar Pichai (Google),Satya Nadella (Microsoft) once arrived to the US on a H1B visa.\
**Motivation**: Our team consists of five international graduate students, in the future we will be applying for H1B visa. The visa application process seems very long, complicated and uncertain. So we decided to understand this process and use Machine learning algorithms to predict the acceptance rate and trends of H1B visa.

In [0]:
from google.colab import drive # mount google drive to access the datasets 
drive.mount('/content/gdrive')

In [0]:
!pip install autocorrect
import string,nltk,warnings,sklearn
import pandas as pd
import numpy as np
import read_files as rd
import visualizations as vz
import plotly.graph_objects as go
import plotly.tools as tls
import cleaning as cw
import baseline as blc
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
from autocorrect import Speller 
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
from collections import Counter
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder #ONE HOT ENCODING
from sklearn.ensemble import RandomForestClassifier #Build model - Random Forest Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gensim.downloader as api
from sklearn.metrics import precision_recall_curve
nltk.download('wordnet')
nltk.download('words')
from sklearn.tree import DecisionTreeClassifier

### Data 
The data used in the project has been collected from <a href="https://www.foreignlaborcert.doleta.gov/performancedata.cfm">the Office of Foreign Labor Certification (OFLC).</a>The Data provides insight into each petition with information such as the Job title, Wage, Employer, Worksite location etc. To get the dataset click on the above link-> click on Disclosure data -> scroll down to H1B data.\


In [0]:
link2015='/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY15_Q4.csv'
link2016='/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY16.csv'
link2017='/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY17.csv'
link2018='/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2018_EOY.csv'
link2019='/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2019.csv'
#read csv fuction to read and concatenate all the datasets
cleaned=rd.read_csv(pd,link2015,link2016,link2017,link2018,link2019)

In [0]:
original=cleaned.copy()
cleaned.shape

### Exploratory Data Analysis
Before we begin working on our data we need to understand the traits of our data which we accomplish using EDA. We see that we have about 260 columns , not all 260 columns have essential information that contributes to our analysis. Hence we pick out the columns such as case status( Accepted/ Denied) ,Employer, Job title etc.We can look at all the colums and the types of object in each column using cleaned.info(). We also use cleaned['column_name'].value_counts() to find the classes in the column along with the count. For feature engineering we are converting quantitative data to categorical.


####Granularity:
Each row in the dataframe represents an application that was filled by the employer and some values are added during the processing of the application such as DECISION_DATE, CASE_STATUS.

In [0]:
cleaned.dropna(inplace=True)
cleaned.shape#shape of the df after dropping all null values

In [0]:
cleaned.head(3)

In [0]:
cleaned['VISA_CLASS'].value_counts()

In [0]:
# the dataset has categories for Visa which are not of use , we require only H1B visa type , hence we drop all records with other visa types
cleaned.drop(labels=cleaned.loc[cleaned['VISA_CLASS']!='H-1B'].index , inplace=True)

In [0]:
cleaned['CASE_STATUS'].value_counts()

In [0]:
#In case status column we can drop withdrawn records and certified-withdrawn records
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='WITHDRAWN'].index , inplace=True)
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='CERTIFIED-WITHDRAWN'].index , inplace=True)
cleaned['CASE_STATUS'].value_counts()

In [0]:
cleaned.info()

####Temporality
The data is available from 9/9/2019 to 1/1/2015

In [0]:
max_date=cleaned.CASE_SUBMITTED.max()	
min_date=cleaned.CASE_SUBMITTED.min()
print("The date in the dataset ranges from "+str(max_date)+" to",min_date)

In [0]:
#the column wages has a mix of both string and float value types and some record have the symbol '$' which we want to remove
cleaned['WAGES']=cleaned['WAGE_RATE_OF_PAY_FROM'].apply(cw.clean_wages).astype('float')

In [0]:
# the wage information that we have available has different unit of pay
cleaned['WAGE_UNIT_OF_PAY'].value_counts()

In [0]:
# we convert the different units of pay to the type 'Year'
cleaned= cw.clean_wageUnit(np,cleaned)
cleaned.drop(columns=['WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY'],axis=1,inplace=True)

In [0]:
#checking if the WAGES column have outliers
print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

We can see that we have large outliers , we can remove them from the data. 

In [0]:
# Remove the outliers.
lowerBound = 0.001
upperBound = 0.95
result = cleaned.WAGES.quantile([lowerBound, upperBound])
cleaned = cleaned[(result.loc[lowerBound] < cleaned.WAGES.values) &\
                  (cleaned.WAGES.values < result.loc[upperBound])]
print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

In [0]:
#Plot for top 10 companies that have the most applications for H1B visa
plt=vz.top_employer(plt,sns,cleaned)
plt.show()

In [0]:
cleaned= cw.clean_states(cleaned)

#### Data Scope
The data available contains applications submitted by Employers from 2015 till 2019. \
Geographical Scope: The WORKSITE_STATE gives information about the state where the primary worksite location. The data set covers all the states of US. 
The Data set also cover different Industries majority of the applications are for IT industry but not limited to IT, it also covers Medical, Pharmacy, Law, Carpentery etc.

In [0]:
df=cleaned.SOC_TITLE.value_counts()
df.head(5)

In [0]:
df=cleaned.WORKSITE_STATE.value_counts()
df.head(5)

####Faithfulness:
Each record in the dataframe represents an application filled by the employer, hence we see a lot of spelling mistakes and some punctutaions like ,(comma) and some numbers in the JOB_TITLE, SOC_TITLE feilds, but there are no inconsistencies and data falsification.

In [0]:
#The Job_title and Soc_title has inconsistencies such as numeric characters and symbols so it has to be cleaned
cleaned['JOB_TITLE']=cleaned.JOB_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['JOB_TITLE']=cleaned['JOB_TITLE'].str.replace(',', '')
cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['SOC_TITLE']=cleaned['SOC_TITLE'].str.replace(',', '')

In [0]:
#Function to clean data columns, to correct spelling mistakes and convert plural words to singular
cleaned=cw.text_clean(cleaned) 

In [0]:
cleaned.SOC_TITLE.value_counts()

The has lot of different subcategories which have less than 15 applications in them which does not show much significance , removing them help us condense the data and take only the significant rows.


In [0]:
cleaned= cw.drop_less_significant(cleaned)
cleaned=cw.cat_to_num(cleaned)

In [0]:
data_scnt,data_anlst,data_eng,mach_learn=cw.data_jobs(cleaned)

## Are you a international student? Are you looking for jobs in Data Science domain?
The below visualization shows median salary for four data science related jobs and the number of visa applications that are submitted for each job. This gives us insights about the jobs available for international students/workers about the job market for Data Science domain. We get to know the number of jobs available for each caegory as well as the median wage for each category. 

In [0]:
f,ax=plt.subplots(figsize=(8,6))
bplot1=plt.boxplot([data_scnt[data_scnt['WAGES']<200000].WAGES,data_anlst[data_anlst['WAGES']<200000].WAGES,data_eng[data_eng['WAGES']<200000].WAGES,mach_learn[mach_learn['WAGES']<200000].WAGES],patch_artist="True")
ax.set_xticklabels(['Data Scientists','Data Analysts','Data Engineer','Machine Learning'],fontsize=15)
ax.set_title('Salary Distribution for jobs in Data Science field in 2019', fontsize=15)
ax.tick_params(labelsize=10)
colors = ['blue','orange', 'green', 'red'] 
for patch, color in zip(bplot1['boxes'], colors): 
  patch.set_facecolor(color)
plt.show()
datajobs=cw.data_concat(pd,data_scnt,data_anlst,data_eng,mach_learn)
ax2=sns.countplot(x="data", data=datajobs)
ax2.set_title('Number of petitions for each jobs in Data Science field in 2019', fontsize=15)
warnings.simplefilter("ignore")

##Job Distribustion in the US for H1b applicants.
The below visualizations gives us insights into the jobs available in USA for every state. H1B applicants can aim for jobs in states with high job openings.As the majority of applicants are from IT domain and software domain we can see that California has the highest number of jobs.  

In [0]:
cleaned=cw.states_vis(cleaned)
df=cleaned.groupby(['CODE']).size().reset_index(name='counts')
fig=vz.USA_map(go,tls,df)
fig.show()

### Comparison of Education levels for some common courses
![picture](https://drive.google.com/uc?id=1J2SP3r-B17JZSRwkCE34O87iBBqx5CMc)

**Features considered for the classifier**

In [0]:
Dataset = cleaned[["CASE_STATUS", "FULL_TIME_POSITION","SOC_TITLE", "WORKSITE_STATE", "WAGES", "H1B_DEPENDENT"]]
Dataset.reset_index(drop = True,inplace = True)

**CONVERTING CATEGORICAL COLUMNS TO NUMERIC COLUMNS**

In [0]:
# Cleaning SOC_TITLE(job_positions) column by removing punctuations
Dataset=cw.fun_punctuation(Dataset)

**If we have more number of unique values in the categorical columns we can use word_vectors to convert them into numeric columns as one-hot encoding fails to convert.**

In [0]:
df = pd.DataFrame(columns = ["job_position"])
Dataset = cw.fun_word_vectors(Dataset)# converting job titles to numeric values using word_vectors.
df["job_position"] = Dataset["SOC_TITLE"]
df["job_position"] 

In [0]:
Dataset.drop(columns = ["SOC_TITLE"],axis=1,inplace=True)
Dataset.reset_index(drop = True, inplace = True)

In [0]:
# one hot encoding to convert states column to numeric values using one hot encoding
Encoding = OneHotEncoder(handle_unknown = 'ignore',sparse = True)
Encoding_df = pd.DataFrame(Encoding.fit_transform(Dataset[["WORKSITE_STATE"]]).toarray())
Dataset = Dataset.join(Encoding_df)

In [0]:
Dataset.drop(columns = ['WORKSITE_STATE'],axis=1,inplace=True)
word_vectors = df[["job_position"]].values
temp = np.vstack(word_vectors.tolist())

In [0]:
# Y represents the output labels
Y = Dataset["CASE_STATUS"].astype(float)
Dataset.drop(columns = ["CASE_STATUS"],inplace = True)

In [0]:
# all_features represent the input labels
other_features = Dataset.iloc[:].values
all_features = np.hstack([temp, other_features])

In [0]:
#Dependent and independent variables
X = all_features


### Baseline classifier


The baseline classifier is done with a basic model. In this case we are taking the mean of the labels ('certified' and 'denied' for H1B visa approvals). It will give us the base accuracy to which we will compare our classifier's accuracy. Our classifier should have a better accuracy than the baseline classifier accuracy.

In [0]:
# This step assigns a binary class label (0 or 1) to each label for H1B visa approval. 
y = Y.to_numpy().astype(int) # Groundtruth labels for the dataset
counts = cleaned['CASE_STATUS'].value_counts()
print(counts)
print('proportion: ', counts[1]/counts[0], ': 1')

In [0]:
accuracies = []
AUCs = []
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)# Testing with K-folds 
for train_idx, test_idx in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx] 
    prediction = blc.run_clasifier(X_train, y_train, X_test, np)
    fold_accuracy = blc.compute_accuracy(y_test, prediction)
    fold_AUC = blc.compute_AUC(y_test, prediction)
    accuracies.append(fold_accuracy)
    if fold_AUC != None: AUCs.append(fold_AUC)
baseline_clasifier_accuracy = mean(accuracies)
print('Baseline accuracy (K-fold): ', baseline_clasifier_accuracy)
print('AUC K-fold: ', mean(AUCs))

The initial accuracy results of the baseline classifier was 0.99. This result is due to the highly imbalanced data with respect to the Case Status, where there are 624682 CERTIFIED applications and 5158 DENIED applications. The proportion of it is 134.02 to 1. Therefore, a performance measure based on the accuracy is not a good one. A better performance measure in imbalanced data is the Area under the ROC Curve (AUC). It meassures the likelihood that given two random points (one from the positive and one from the negative class) the classifier will rank the point from the positive class higher than the one from the negative one.

### Balancing data

Given the inbalanced data we have, we lack observations required to train an effective model that classifies the status of the H1B visa. To addess that, a good idea is to upsample the minority class and undersample the majority class. First, the ADASYN algorithm is applied to upsample the minority class. It generates new related samples and some random samples to avoid linear correlation to the parent. Second, the ENN algorithm is applied to undersample the majority class. It focuses on removing the noisy observations. The expected proportion between the two classes is 1 to 2 at most.

In [0]:
# Oversampling the minority class
ada = ADASYN(sampling_strategy=0.5, random_state=0)
X_ada, y_ada = ada.fit_resample(X, Y)

In [0]:
# Undersampling the majority class
enn = EditedNearestNeighbours()
X, Y = enn.fit_resample(X_ada, y_ada)

In [0]:
count_y_ada_enn=Counter(Y)
print('Balanced dataset shape %s' % count_y_ada_enn)
print('proportion: ', count_y_ada_enn[1]/count_y_ada_enn[0])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)# Testing with regular split
prediction_bl = blc.run_clasifier(X_train, y_train, X_test, np)
split_accuracy = blc.compute_accuracy(y_test, prediction_bl)
split_AUC = blc.compute_AUC(y_test, prediction_bl)
print('Baseline accuracy (split): ', split_accuracy)  
print('AUC split (80-20): ', split_AUC)

In [0]:
# Confusion matrix
tn, fp, fn, tp = metrics.confusion_matrix(y_test, prediction_bl).ravel()
print("tn, fp, fn, tp", (tn, fp, fn, tp))

In [0]:
# classification report
report = metrics.classification_report(y_test, prediction_bl)
print(report)

In [0]:
# Plot precision recall curve
plt=vz.plot_precision_recall(plt,y_test, prediction_bl,precision_recall_curve)

### Random forest

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=20)
model_clf=RandomForestClassifier(n_jobs=2,random_state=0)
model_clf.fit(X_train,y_train)#train the model
prediction_test = model_clf.predict(X_test)#test the model (predict with our test data)
prediction_test

In [0]:
#compare with original value, Y_test
print("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))

In [0]:
# Confusion matrix
metrics.confusion_matrix(y_test, prediction_test)

In [0]:
# classification report
report = metrics.classification_report(y_test, prediction_test)
report

In [0]:
y_probs_rand_forest = model_clf.predict_proba(X_test)
vz.plot_precision_recall(plt, y_test, y_probs_rand_forest)

##### **President Trump's "Buy American and Hire American" Executive order's impact on H1B visa**
The "Buy American and Hire American" order was passed on April 18, 2017, to Order to protect American workers and strengthen the American economy. Among other purposes, it aims to detect and prevent fraud in the process. Also, to avoid the abuse of the program which is heavily used by technology companies. This statement is interesting to evaluate from our perspective as international students.
The after-effects of which caused a gradual decrease in the request and acceptance of H1B visas have been creating more job opportunities for immigrants which in turn contributes to the American workforce.

It was also observed that just before this order was passed there was a surge in the applications submitted. 

As the graph demonstrates, this type of order, after it has been approved, take some months (around 6 months in this case)  to show the after-effects. It's in 2018 when we see the number of applications and their acceptance is falling so far, as shown in the below visualization.

In [0]:
plt=vz.plot_buy_american_order(plt, original, pd)
plt.show()

From the above visualization made on a random sample of about 20000 records from 2015 to 2019, we can infer that the visa acceptance is gradually seen as a downfall, which is one the after-effects of the executive order passed earlier 2017.


In [0]:
#mapping wages to 4 categories
cleaned['WAGE_CATEGORY'] =pd.cut(cleaned.WAGES,bins=[0,50000,100000,140000,200000],
                                 labels=['LOW','AVERAGE','HIGH',"VERY HIGH"])
df_temp = pd.DataFrame(columns = ["WC_NUM"])
cleaned['WC_NUM'] = pd.cut(cleaned.WAGES,bins=[0,50000,100000,140000,200000],labels=[0,1,2,3])
df_temp["WC_NUM"]=pd.cut(cleaned.WAGES,bins=[0,50000,100000,140000,200000],labels=[0,1,2,3])

### **WAGE RANGE RANGE PREDICTION:**
Using the data from our dataset we divide the wages into 4 categories which is LOW, AVERAGE, HIGH, VERY_HIGH.
And this predictor will predict the expected salary/wage range for any H1B visa holder or an H1B aspirant based on various factors like location , occupation ,employer etc.


**BASELINE CLASSIFIER FOR WAGE RANGE PREDICTOR:**

In [0]:
#assigning X, y for baseline classifier wage classifier.
X = cleaned['WC_NUM'].to_numpy()
y = df_temp["WC_NUM"].to_numpy().astype(int)

In [0]:
accuracies = []
AUCs = []
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)# Testing with K-folds 
for train_idx, test_idx in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx] 
    prediction = blc.run_clasifier(X_train, y_train, X_test, np)
    fold_accuracy = blc.compute_accuracy(y_test, prediction)
    fold_AUC = blc.compute_AUC(y_test, prediction)
    accuracies.append(fold_accuracy)
    if fold_AUC != None: AUCs.append(fold_AUC)
baseline_clasifier_accuracy = mean(accuracies)
print('Baseline accuracy (K-fold): ', baseline_clasifier_accuracy)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)# Testing with regular split
prediction = blc.run_clasifier(X_train, y_train, X_test, np)
split_accuracy = blc.compute_accuracy(y_test, prediction)
#split_AUC = multiclass_roc_auc_score(y_test, prediction)
print('Baseline accuracy (split): ', split_accuracy)

**Features considered for the wage classifier**

In [0]:
order_date = '01-01-2018'
cleaned['CASE_SUBMITTED']=pd.to_datetime(cleaned['CASE_SUBMITTED'])
mask1 = (cleaned['CASE_SUBMITTED'] < order_date)
cleaned_before = cleaned.loc[mask1]
Dataset_wage=cleaned_before[["EMPLOYER_NAME","FULL_TIME_POSITION","SOC_TITLE", "WORKSITE_STATE", "WC_NUM","CASE_STATUS"]]
Dataset_wage.reset_index(drop = True,inplace = True)

In [0]:
# Considering top 70 employers and remaining employers as others
Top_EMPLOYER = Dataset_wage["EMPLOYER_NAME"].value_counts().head(70)
def emp_function(emp):
    if emp in Top_EMPLOYER:
        return emp
    else:
        return "others"
Dataset_wage["TOP_EMPLOYER"] = Dataset_wage["EMPLOYER_NAME"].apply(emp_function)
Dataset_wage.drop(columns=["EMPLOYER_NAME"],axis = 1,inplace=True)

In [0]:
Dataset_wage["SOC_TITLE"] = Dataset_wage["SOC_TITLE"].apply(lambda x : cw.remove_punctuation(x))
df_wage = pd.DataFrame(columns = ["soc_title"])
Dataset_wage = cw.fun_word_vectors(Dataset_wage)# converting job titles to numeric values using word_vectors.
df_wage["soc_title"] = Dataset_wage["SOC_TITLE"]

In [0]:
df_wage["soc_title"]

In [0]:
Dataset_wage.drop(columns=["SOC_TITLE"],axis=1,inplace=True)
Dataset_wage.reset_index(drop = True, inplace = True)

In [0]:
Encoding = OneHotEncoder(handle_unknown='ignore',sparse = True)
Encoding_wage = pd.DataFrame(Encoding.fit_transform(Dataset_wage[["WORKSITE_STATE","TOP_EMPLOYER"]]).toarray())
Dataset_wage = Dataset_wage.join(Encoding_wage)
Dataset_wage.drop(columns=["WORKSITE_STATE","TOP_EMPLOYER"],axis=1,inplace=True)

In [0]:
Dataset_wage.head()

In [0]:
wv_wage = df_wage[["soc_title"]].values
temp_var = np.vstack(wv_wage.tolist())

In [0]:
Y = Dataset_wage["WC_NUM"].astype(float)
Dataset_wage.drop(columns = ["WC_NUM"],inplace = True)
# sum_features represent the input data 
other_features = Dataset_wage.iloc[:].values
del Dataset_wage
other_features.shape

In [0]:
sum_features = np.hstack([temp_var, other_features])
X = sum_features

In [0]:
#Split the date into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=20)

In [0]:
dec_clf = DecisionTreeClassifier(max_depth=15,criterion='gini')
dec_clf.fit(X_train,y_train)#train the model
prediction_test = dec_clf.predict(X_test)#test the model (predict with our test data)

In [0]:
print("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))#compare with original value, Y_test

In [0]:
metrics.confusion_matrix(y_test, prediction_test)

In [0]:
report = metrics.classification_report(y_test, prediction_test)
print(report)